In [1]:
#Paqueterias necesarias para la realizacion de todo el códigatzo
import numpy as np
import math
from copy import copy
import random
import statistics
from random import randrange

#Solo necesitas meter el txt en la celda de aquí abajo. 
(De favor, el txt debe estar como en el ejemplo de entrada en el manual de metaheuristica). Sino, te tocará meter las entradas a manita xd

In [2]:
rs = "/content/drive/MyDrive/Computo evolutivo/RecocidoSimulado/mochila.txt"

#Poner el número de ejecuciones que deseas. En mi caso, le puse 6

In [3]:
M = 9

#Lector del txt, y organizador las entradas para poder ser compiladas en las funciones

Se asume que solo la temperatura puede ser flotante; Me faltó asumir que en la búsqueda tabú todas las entradas eran enteros.
Sino, pues se cambian los int() por float(), como lo hice con la temperatura

In [4]:
def mochilita(texto):
  with open(texto) as f: #Abro el txt
    contents = f.readlines() #Leo el contenido del mismo


  #Temperatura
  te = '' #Esta es la herramienta mágica :vvv
  temp = [] #Lista vacía donde se van a guardar las temperaturas

  for i in range(len(contents[0])): #Como vienen las temperaturas iniciales y finales en la misma línea
      if contents[0][i] != ' ' and contents[0][i] != '\n': #Si es diferente a un espacio o un salto de línea
        te += contents[0][i] #Añade ese texto a la herramienta mágica.
      else:
        te = float(te) #Convierte el texto a entero
        temp.append(te) #Metelo en la lista de costos
        te = '' #Vacía la herramienta mágica


  #Número de objetos
  obj = contents[1][:-1]
  obj = int(obj)


  #Capacidad de la mochila
  moch = contents[2][:-1]
  moch = int(moch)


  #Valor de pi y wi de cada objeto i
  ahsu = '' #Otra herramienta mágica ;v
  costs = [] #Lista donde voy a guardar los pi y wi

  if contents[-1][-1] != '\n': #Si contents en la última fila es diferente de un salto de línea
    contents[-1] += '\n' #Añadiendo salto de línea, porque en la última fila no se ecuentra

  for i in range(3,len(contents)): #Como las primeras tres líneas corresponden a entradas diferentes, empiezo en tres. Porque el contenido de aquí para abajo es de la entrada 4
      for j in range(len(contents[i])): #Recorre la fila i
        if contents[i][j] != ' ' and contents[i][j] != '\n': #Si es diferente a un espacio o un salto de línea
          ahsu += contents[i][j] #Añade ese texto a la herramienta mágica.
        else:
          ahsu = int(ahsu) #Convierte el texto a entero
          costs.append(ahsu) #Metelo en la lista de costos
          ahsu = '' #Vacía la herramienta mágica


  #Lista con las pi
  pi = []

  for p in range(0, len(costs), 2):
    pi.append(costs[p])

  
  #Listca con las wi
  wi = []

  for w in range(1, len(costs), 2):
    wi.append(costs[w])


  return temp, obj, moch, pi, wi

In [5]:
entradas = mochilita(rs); entradas

([1000.0, 0.1], 5, 15, [5, 14, 7, 2, 23], [2, 3, 7, 5, 10])

In [6]:
#Temperatura inicial
tem_ini = entradas[0][0]; tem_ini

1000.0

In [7]:
#Temperatura final
tem_fin = entradas[0][1]; tem_fin

0.1

In [8]:
#Objetos N
obj_n = entradas[1]; obj_n

5

In [9]:
#Capacidad de la mochila
cap_moch = entradas[2]; cap_moch

15

In [10]:
#Valor pi de cada objeto i
val_pi = entradas[3]; val_pi

[5, 14, 7, 2, 23]

In [11]:
#Peso wi de cada objeto i
peso_wi = entradas[4]; peso_wi

[2, 3, 7, 5, 10]

#Generar solución inicial aleatoriamente

In [12]:
def sol_inicial(obj_n, cap_moch, val_pi, peso_wi):
  sol = [0]*obj_n #Creando lista con la longitud de los objetos N. Como todavía no hay ningún objeto en la mochila, todos son 0.
  valor = 0 #Valor de la solución inicial
  peso = 0 #Peso de la solución inicial
  ya_vis = [] #Lista donde voy a guardar los objetos i ya explorados, para evitar que se cicle infinitamente
  obj_moch = [] #Lista donde van los objetos que pudieron entrar en la mochila

  while peso < cap_moch and len(ya_vis) < len(sol): #Mientras el peso sea menor a la capacidad de la mochila y 
    ale = randrange(obj_n) #Generar un número aleatorio de 0 a n-1
    while sol[ale] == 1: #Mientras el número generado ya esté en la mochila
      ale = randrange(obj_n) #Generar un número aleatorio de 0 a n-1
    else: #Si ese número generado no está en la mochila
      if ale not in ya_vis: #Si ese número generado no se encuentra en la lista donde ya se exploraron los objetos i
        peso += peso_wi[ale] #Sumarle el peso del objeto
        if peso <= cap_moch: #Si la suma no excede la capacidad de la mochila
          valor += val_pi[ale] #Sumar el valor del objeto
          sol[ale] = 1 #Cambiar a 1 en la lista de longitud de los objetos N. Porque ese objeto ahora está dentro de la mochila
          obj_moch.append(ale) #Insertar el objeto a la lista donde se encuentran los objetos que estarán en la mochila
          ya_vis.append(ale) #Insertar el número generado
        else: #Si la suma del peso con el objeto elegido supera la capacidad de la mochila
          peso -= peso_wi[ale] #Restarle el peso del objeto
          ya_vis.append(ale) #Insertar el objeto i, porque ya se sabe que no puede entrar en la mochila.

  return obj_moch, sol, valor, peso

sol_ini = sol_inicial(obj_n, cap_moch, val_pi, peso_wi); sol_ini

([4, 0, 1], [1, 1, 0, 0, 1], 42, 15)

#Generar vecindario

In [13]:
def vec(sol_ini, peso_wi, cap_moch):

  copia_sol_bin_ini = sol_ini[1].copy() #Hago una copia de la permutación binaria
  vecindario = [] #Lista vacía donde guardaré las soluciones factibles


  for i in range(len(copia_sol_bin_ini)): #Recorrer la lista binaria
    if sol_ini[1][i] == 1: #Si la solución inicial (Que en el código donde se junta todo será la mejor solución hasta ahora) en la lista binaria en la posición i es 1
      copia_sol_bin_ini[i] = 0 #Cambiar su valor de 0 a 1
    else:
      copia_sol_bin_ini[i] = 1 #Cambiarlo a 1, porque su valor era 0

    peso = 0
    for p in range(len(copia_sol_bin_ini)): #Recorrer la nueva lista binaria, donde se cambio ya sea un 1 por 0 o 0 por 1
      if copia_sol_bin_ini[p] == 1: #Si es uno, significa que está adentro de la mochila
        peso += peso_wi[p] #Sumarle su peso al peso total (peso)
      else:
        None #Si es 0, pues no hagas nada xd

    if 0 < peso <= cap_moch: #Si la mochila al menos tiene un objeto adentro, su peso es mayor a 0, y si su peso no supera la capacidad de la mochila, es factible
      vecindario.append(copia_sol_bin_ini) #Insertar esa permutación binaria a la lista de los vecindarios
      copia_sol_bin_ini = sol_ini[1].copy() #Regresar la copia de la lista binaria a la permutación original
    else:
      copia_sol_bin_ini = sol_ini[1].copy() #No jaló, toncs pues solo vuelve a ser la permutación original

  return vecindario #Regresa el vecindario con las soluciones factibles

In [14]:
vecindario = vec(sol_ini, peso_wi, cap_moch); vecindario

[[0, 1, 0, 0, 1], [1, 0, 0, 0, 1], [1, 1, 0, 0, 0]]

#Escoger aleatoriamente solución dentro del vecindario

In [15]:
def ale_vec(vecindario, val_pi, peso_wi, Tt, mejor_sol):
  sele_ale = randrange(len(vecindario)) #Elegir un número entero que vaya de 0 a n-1. Para elegir alguna solución del vecindario de manera aleatoria

  
  #Conseguir el valor de los objetos que están dentro de la mochila
  valor = 0
  for v in range(len(vecindario[sele_ale])):
      if vecindario[sele_ale][v] == 1:
        valor += val_pi[v]
      else:
        None


  #Peso en conjunto de los objetos en la mochila
  peso_sele = 0
  for pe in range(len(vecindario[sele_ale])):
      if vecindario[sele_ale][pe] == 1:
        peso_sele += peso_wi[pe]
      else:
        None


  obj_moch = [] #Lista donde podrían estar los objetos que estarán en la mochila
  if valor > sol_ini[2]: #Si el valor de esos objetos es mayor al mayor valor hasta ahora
    for o in range(len(vecindario[sele_ale])): #Conseguir que objetos son, y meterlos a la lista
      if vecindario[sele_ale][o] == 1:
        obj_moch.append(o)
      else:
        None
    mejor_sol = [obj_moch, vecindario[sele_ale], valor, peso_sele] #Actualizar la mejor solución
  elif random.uniform(0, 1) < math.exp(-1.*(valor - sol_ini[2])/Tt): #Si el número random es menor que esa exponencial xd
    for o in range(len(vecindario[sele_ale])): #Conseguir que objetos son, y meterlos a la lista
      if vecindario[sele_ale][o] == 1:
        obj_moch.append(o)
      else:
        None
    mejor_sol = [obj_moch, vecindario[sele_ale], valor, peso_sele] #Actualizar la mejor solución
  Tt *= 0.99 #Multiplicar la temperatura actual por 0.99, para ser actualizada

  return mejor_sol

In [16]:
ale_vec(vecindario, val_pi, peso_wi, tem_ini, sol_ini)

[[0, 4], [1, 0, 0, 0, 1], 28, 12]

#Mientars Tt > Tf (Juntando los dos códigos de arriba)

Ya no lo comento, porque todo se encuentra descrito arriba

In [17]:
def v(peso_wi, cap_moch, val_pi, Tt, Tf, mejor_sol):

  while Tt > Tf: #Mientras la temperatura actual sea mayor a la temperatura final
    copia_sol_bin_ini = mejor_sol[1].copy()
    vecindario = []


    for i in range(len(copia_sol_bin_ini)):
      if mejor_sol[1][i] == 1:
        copia_sol_bin_ini[i] = 0
      else:
        copia_sol_bin_ini[i] = 1

      peso = 0
      for p in range(len(copia_sol_bin_ini)):
        if copia_sol_bin_ini[p] == 1:
          peso += peso_wi[p]
        else:
          None

      if 0 < peso <= cap_moch:
        vecindario.append(copia_sol_bin_ini)
        copia_sol_bin_ini = mejor_sol[1].copy()
      else:
        copia_sol_bin_ini = mejor_sol[1].copy()


    sele_ale = randrange(len(vecindario))

    
    valor = 0
    for v in range(len(vecindario[sele_ale])):
        if vecindario[sele_ale][v] == 1:
          valor += val_pi[v]
        else:
          None


    peso_sele = 0
    for pe in range(len(vecindario[sele_ale])):
        if vecindario[sele_ale][pe] == 1:
          peso_sele += peso_wi[pe]
        else:
          None


    obj_moch = []
    if valor > mejor_sol[2]:
      for o in range(len(vecindario[sele_ale])):
        if vecindario[sele_ale][o] == 1:
          obj_moch.append(o)
        else:
          None
      mejor_sol = [obj_moch, vecindario[sele_ale], valor, peso_sele]
    elif random.uniform(0, 1) < math.exp(-1.*(valor - mejor_sol[2])/Tt):
      for o in range(len(vecindario[sele_ale])):
        if vecindario[sele_ale][o] == 1:
          obj_moch.append(o)
        else:
          None
      mejor_sol = [obj_moch, vecindario[sele_ale], valor, peso_sele]
    else:
      None
    Tt *= 0.99

  return mejor_sol

v(peso_wi, cap_moch, val_pi, tem_ini, tem_fin, sol_ini)

[[1, 4], [0, 1, 0, 0, 1], 37, 13]

#Realización de las M ejecuciones

Me disculpo si son inconsistentes los comentarios, hice rechicken de la práctica pasada, solo le realicé cambios minimos xd

In [18]:
#Mejor solución encontrada en las M ejecuciones
def mejor_sol_M(M):
  bests = [v(peso_wi, cap_moch, val_pi, tem_ini, tem_fin, sol_ini) for _ in range(M)] #Correr M veces la función donde se obtiene la mejor mochila
  

  #Mejor y 'pior' solución
  best = [] #Lista donde voy a guardar la mejor solución
  worst = [] #Lista donde voy a guardar la peor solución
  costes = [] #Lista donde voy a guardar los costos
  for b in range(len(bests)): #Correr esto M veces
    costes.append(bests[b][2]) #Meter en la lista de costes, el valor de la permutación en la posición b
    #print(costes)
  diccio = { i : costes[i] for i in range(0, len(costes) ) } #Diccionario de los valores
  #print(diccio)
  mejor_permu = max(diccio, key=diccio.get) #Obtener la llave de la permutación con mayor valor
  pior_permu = min(diccio, key=diccio.get) #Obtener la llave de la permutación con menor valor
  best.append(bests[mejor_permu]) #Agregar la mejor permutación con su valor
  worst.append(bests[pior_permu]) #Agregar la pior permutación con su valor


  #Mediana
  elementos = [] #Lista donde se va a enumerar los números correspondiente a la longitud de M
  mediana_permu = [] #Número o lista, dependiendo si es par o impar M
  median = [] #Lista donde voy a guardar la o las mediana(s)
  orden = sorted(diccio.items(), key=lambda x:x[1]) #Ordenar con respecto a los valores del diccionario
  #print(orden)

  for me in range(len(orden)): #Correr esto M veces
    elementos.append(me) #Agregar de manera ascendente número de 0 a M

  if len(elementos)%2 != 0: #Si es impar
    per_med = np.median(np.array(elementos)) #Sacar la mediana
    per_med = int(per_med) #Volverlo entero
    mediana_permu = orden[per_med][0] #Obtener la llave que está en la posicón [per_med], o sea, la mediana
    median.append(bests[mediana_permu]) #Agregar a la lista de la mediana la permutación, su valor y su peso
  else: #Si es par
    per_med = np.median(np.array(elementos)) #Sacar la mediana
    per_med_izq = per_med//1 #Hacer piso. Este número va a corresponder a la mediana en la posición izquierda
    per_med_izq = int(per_med_izq) #Convertir a entero
    per_med_der = per_med+.5 #Sumarle .5 para que corresponda a la mediana de la posición derecha
    per_med_der = int(per_med_der) #Volverlo entero
    mediana_permu.append(orden[per_med_izq][0]) #Agregar a la lista la llave que está en la posicón [per_med_izq], o sea, la mediana izquierda
    mediana_permu.append(orden[per_med_der][0]) #Hacer lo mismo pero con la mediana derecha
    median.append(bests[mediana_permu[0]]) #Agregar a la lista de la mediana la permutación izquierda, su valor y su peso.
    median.append(bests[mediana_permu[1]]) #Agregar a la lista de la mediana la permutación derecha, su valor y su peso.


  #Media
  mean_costs = [] #Lista donde voy a guardar los valores de cada permutación obtenida de las ejecuciones.

  for mc in range(len(orden)): #Correr esto M veces
    mean_costs.append(orden[mc][1]) #Agregar el valor en la posición [mc] a la lista

  mean = sum(mean_costs)/len(mean_costs) #Sacar el promedio de los costos


  #Desviación estándar
  standard_deviation = statistics.stdev(mean_costs) #Desviacíon estándar de los costos de las M permutaciones


  #Para que veas cada una de las ejecuciones, por si crees que te estoy chamaqueando :vvv
  #print(bests,'\n''Mejor solución: ', best,'\n''Peor solución: ', worst,'\n''Mediana(s): ', median,'\n''Media: ', mean, '\n''Desviación estándar: ', standard_deviation)
  

  print('Mejor solución: ', best,'\n''Peor solución: ', worst,'\n''Mediana(s): ', median,'\n''Media: ', mean, '\n''Desviación estándar: ', standard_deviation)


mejor_sol_M(M)

Mejor solución:  [[[1, 4], [0, 1, 0, 0, 1], 37, 13]] 
Peor solución:  [[[0, 3], [1, 0, 0, 1, 0], 7, 7]] 
Mediana(s):  [[[1, 2], [0, 1, 1, 0, 0], 21, 10]] 
Media:  22.666666666666668 
Desviación estándar:  9.886859966642595
